In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from tqdm import tqdm_notebook as tqdm

my_env = 'data'
my_key = get_key('koray_data')
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=FileProcessed"
all_pc_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_pc_resp))

469


In [ ]:
# find all runs from 4dn lab without a contributing lab
files_with_source = [i for i in all_pc_resp if i.get('source_experiments')]
print(len(files_with_source))
## Patching all processed files
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
action = True
def get_attr_from_pc_file(file_resp):
    lab = ''
    award = ''
    source_exp = file_resp['source_experiments'][0]
    exp_info = ff_utils.get_metadata(source_exp, key=my_key, add_on='frame=raw')
    lab = exp_info['lab']
    award = exp_info['award']
    cont_labs = exp_info.get('contributing_labs', [])
    return lab, award, cont_labs

counter = 0
for a_file in files_with_source:
    counter += 1
    if counter % 100 == 0:
        print(counter)
    my_lab, my_award, my_cont_labs = get_attr_from_pc_file(a_file)
    if my_lab in my_cont_labs:
        print("PROBLEM, owner lab also in cont_labs", skipping)
        continue
    if FDlab not in my_cont_labs:
        my_cont_labs.append(FDlab)
    patch_data = {'lab':my_lab,
                  'award': my_award,
                  'contributing_labs':my_cont_labs}
    #print(patch_data)
    if action:
        ff_utils.patch_metadata(patch_data,obj_id=a_file['uuid'], key=my_key)

In [ ]:
# find all runs from 4dn lab
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=WorkflowRunAwsem"
all_awsem_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_awsem_resp))

In [ ]:
files_with_cont_and_lab_same = []
# add lab/award/contlab to wfrs
counter = 0
action = True
acted = 0
for a_resp in all_awsem_resp: 
    counter += 1
    if counter % 100 == 0:
        print(counter, acted)
    
    my_lab = ''
    my_award = ''
    my_cont_lab = ''
    patch_data = {}
    
    input_files = a_resp['input_files']
    run_on_expected = False
    for an_input in input_files:
        # if this wfr ran on processed files
        input_id = an_input['value']['@id']
        if input_id.startswith('/files-processed/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key=my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
        # if this wfr ran on raw files
        elif input_id.startswith('/files-fastq/') or input_id.startswith('/files-microscopy/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key = my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
    # I normally exclude files reference since they are part of many runs, and often belong to dcic
    # in a second round, if the file type is not in one if the expected ones, check what it is 
    # this case often convers md5 runs on reference files or other file types (eg filecalibration)
    if not run_on_expected:
        print('skipping wfr on', input_id)
        continue

    if my_lab == FDlab:
        continue
           
    # if file is not from FDN(user submitted pc), add it to cont labs
    if FDlab not in my_cont_lab:
        my_cont_lab.append(FDlab)  

    if my_lab in my_cont_lab:
        my_cont_lab.remove(my_lab)
        files_with_cont_and_lab_same.append(input_id)
        print(an_input['value']['accession'], 'has lab also in cont lab')
        
    patch_data = {'lab':my_lab,
                  'award': my_award}
    if my_cont_lab:
        patch_data['contributing_labs'] = my_cont_lab
    if action:
        ff_utils.patch_metadata(patch_data,obj_id=a_resp['@id'], key=my_key)
        acted += 1



In [3]:
# find all qc from 4dn lab
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=QualityMetric"
all_qc_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_qc_resp))

117


In [4]:
from tqdm import tqdm_notebook as tqdm
it = 0
changed = 0
not_ass = []
action = True
for qc in tqdm(all_qc_resp):
  
    it += 1
    if it % 100 == 0:
        print(it, changed, len(not_ass))
    ac = ''
    try:
        url = qc['url']
        acs = [i[:12] for i in url.split('/') if i.startswith('4DN')]
        if len(acs) != 1:
            print(adana)
        else:
            ac = acs[0]
    except: 
        links = ff_utils.get_metadata(qc['@id']+'@@links', my_key).get('uuids_linking_to', [])
        files = []
        if links:
            files = [i['@id'] for i in links if 'file' in i['@id']]
        else:
            print('can not find associated files')
            not_ass.append(qc['uuid'])
            continue

        if len(files) > 1:
            print('multiple files associated, skipping')
            continue
        ac = files[0].split('/')[2]
        
    if not ac.startswith('4DN'):
        print(ac)
        print(qc['uuid'])
        break
        
        
    try:
        file_info = ff_utils.get_metadata(ac, my_key, add_on = 'frame=raw')
    except:
        print(ac,'can not find, from qc', qc['uuid'] )
    lab = file_info['lab']
    if lab == '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989':
        continue
        
    award = file_info['award']
    cont_labs = file_info.get('contributing_labs', [])

    patch_data = {'lab':lab,
                  'award': award}
    if cont_labs:
        patch_data['contributing_labs'] = cont_labs
    changed += 1
    if action:
        ff_utils.patch_metadata(patch_data, obj_id = qc['uuid'], key=my_key)

 
print(len(not_ass), 'files not associated with qc')    

100 89 0

0 files not associated with qc


In [17]:
print(not_ass)

['73ffbf1d-e446-428e-8630-cb1ea0ce3c21', '923b8cb3-d85e-47ef-84bc-8a32b7e4b7ef', '131106bc-8535-4448-903e-854abbbbbbbb', '131106bc-8535-4448-903e-8577f460bcc0']


In [23]:
del_ids =  ['131106bc-8535-4448-903e-854af460b000', '73ffbf1d-e446-428e-8630-cb1ea0ce3c21', '923b8cb3-d85e-47ef-84bc-8a32b7e4b7ef', '131106bc-8535-4448-903e-854abbbbbbbb', '131106bc-8535-4448-903e-8577f460bcc0']
for an_id in del_ids:
    resp = ff_utils.get_metadata(an_id, my_key)
    print(resp['display_title'], resp['status'])
    resplink = ff_utils.get_metadata(resp['@id']+'@@links', my_key).get('uuids_linking_to', [])
    if resplink:
        link_ids = [i['@id'] for i in resplink]
        for link_id in link_ids:
            resp2 = ff_utils.get_metadata(link_id, my_key)
            print(resp2['display_title'], resp2['status'])
        print('!!! linked items', )
    ff_utils.patch_metadata({'status': 'deleted'}, obj_id = an_id, key=my_key)
        

QualityMetricFastqc from 2017-08-10 in review by lab
WorkflowRunSbg from 2017-08-10 deleted
WorkflowRunAwsem from 2017-09-07 deleted
WorkflowRun from 2017-08-10 deleted
!!! linked items
QualityMetricDedupqcRepliseq from 2018-08-14 in review by lab
QualityMetricDedupqcRepliseq from 2018-08-14 in review by lab
QualityMetricPairsqc from 2017-08-10 in review by lab
QualityMetricBamqc from 2017-08-10 in review by lab
